In [1]:
from langchain.prompts import ChatPromptTemplate
from langchain_ollama import OllamaLLM
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda

#model = OllamaLLM(model="llama3.2:latest")
model = OllamaLLM(model="gemma3:4b")

In [2]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system","你是一個專業的客服代表，負責回覆客戶郵件。請用友善和專業的語調回覆。請不要回覆(公司的資訊和結尾詞),我會自已手動增加"),
    ("human","請回覆這封客戶郵件：\n\n{email_content}")
])

In [3]:
def format_email_reply(ai_answer:str) -> str:
    import datetime
    date_str = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
    formatted_reply = f"""{ai_answer}

感謝您的來信，如有其他問題請隨時聯繫我們。

此致
致理客服團隊

回覆時間:{date_str}
"""

    print(f"ai回覆的內容:{formatted_reply}")

    return {"formatted_reply":formatted_reply}

format_email = RunnableLambda(format_email_reply)

In [4]:
improve_prompt = ChatPromptTemplate.from_messages(
    [
        ("system","你是郵件品質專家。請評估這封郵件的專業度、友善度和完整度（1-10分），並提供具體的改進建議。"),
        ("human","估並改進這封郵件:\n\n{formatted_reply}")
    ]
)

In [5]:
chain = (prompt_template 
        | model
        | StrOutputParser()
        | format_email
        | improve_prompt
        | model
        | StrOutputParser()
)

In [6]:
customer_email = """您好:

我最近購買了貴公司的產品，但是發現包裝有損壞，而且產品功能與說明書不符。
請問可以退貨或換貨嗎？我已經購買一週了。

期待您的回覆。


王小明"""

In [7]:
result = chain.invoke({
    "email_content":customer_email
})

print(result)

ai回覆的內容:王小明您好，

非常抱歉讓您有不愉快的購物體驗。我們深感遺憾聽到您收到的產品狀況與說明書不符，以及包裝的損壞。

請您放心，我們會盡全力協助您解決問題。請您提供更多關於產品狀況的詳細資訊，例如：

*   產品型號
*   損壞的具體情況（請附上照片或影片）
*   功能不符的具體描述

我們將會審視您的情況，並在收到您的資訊後盡快與您聯繫，討論最佳的退貨或換貨方案。

再次向您致以歉意，感謝您的理解與耐心。

感謝您的來信，如有其他問題請隨時聯繫我們。

此致
致理客服團隊

回覆時間:2025-10-18 15:27

好的，我來為這封郵件進行專業度、友善度和完整度的評估，並提供具體改進建議：

**評估：**

*   **專業度： 8/10** - 語氣和措辭基本專業，但可以更精準地表達對問題的重視。
*   **友善度： 9/10** - 開頭道歉、表達關心，並提供協助，整體語氣友善，能讓客戶感到被重視。
*   **完整度： 6/10** - 雖然詢問了基本資訊，但可以更全面地讓客戶了解解決流程和時間表，增加客戶信心。

**具體改進建議：**

1.  **更明確表達關心，開頭更正式：**
    *   改寫開頭： “王小明先生/女士，您好。非常抱歉讓您有不愉快的購物體驗，我們深感遺憾，並真誠地為您帶來的不便致歉。” （使用更正式的稱謂，並明確表達歉意）

2.  **強化問題解決的流程：**
    *   可以更清楚地說明接下來的處理流程，例如： “收到您的資訊後，我們會 [X個工作天] 內進行審核，並將 [負責人姓名/職稱] 與您聯繫，討論最佳的退貨或換貨方案。同時，我們也會 [說明處理流程，例如：為您安排寄回貨運/提供退貨地址/安排技術支援]。 ” （提供更具體的時間表和處理流程，建立客戶信心）
    *   建議加入： “為了加快處理速度，請您提供以下資訊：…”

3.  **更具體地提出資訊需求：**
    *   將“功能不符的具體描述”更明確地問清楚，例如：“請詳細描述您遇到的功能不符情況，並提供具體的測試步驟/結果。” (建議附上客戶可以參考的常見問題解答連結，幫助客戶更準確描述問題)
    *   可以考慮提供一個表格讓客戶更容易填寫資訊。

4.  **提升結尾的專業度：**
    *   改寫結尾：“再